In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [3]:
train_nlu_model_switch = True

In [50]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    2.5s finished


In [26]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [51]:
query = 'i own the status of an nr what are the categories of taxable income for me'

In [52]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.701850072056577,
   'end': 25,
   'entity': 'status',
   'extractor': 'ner_crf',
   'start': 23,
   'value': 'nr'},
  {'confidence': 0.9109174793392143,
   'end': 67,
   'entity': 'subject',
   'extractor': 'ner_crf',
   'start': 61,
   'value': 'income'}],
 'intent': {'confidence': 0.53783608203988309, 'name': 'types'},
 'intent_ranking': [{'confidence': 0.53783608203988309, 'name': 'types'},
  {'confidence': 0.057592848350408482, 'name': 'conditions'},
  {'confidence': 0.057201588423080825, 'name': 'purpose'},
  {'confidence': 0.056526862802790824, 'name': 'definition'},
  {'confidence': 0.053517775282787394, 'name': 'benefits'},
  {'confidence': 0.053384362114136559, 'name': 'procedure'},
  {'confidence': 0.034650300127331161, 'name': 'address'},
  {'confidence': 0.029894270508490361, 'name': 'duration'},
  {'confidence': 0.02224245809013594, 'name': 'contact'},
  {'confidence': 0.020924087955223297, 'name': 'legislation'}],
 'text': 'i own the status 